**Import libraries**

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

**Read the data**

In [ ]:
data = open('data.txt', 'r').read().splitlines()
data = [str(line.encode('ascii', 'ignore'), 'utf-8') for line in data]

**Char/idx - idx/char mapping**

In [ ]:
char = list(set(''.join(data)))
char_to_idx = {char: idx+1 for idx, char in enumerate(sorted(char))}
char_to_idx['\n'] = 0
idx_to_char = {v: k for k, v in char_to_idx.items()}

**Vocabulary size**

In [ ]:
vocab_size = len(idx_to_char)

**Prepare data**

In [ ]:
data = ''.join([line+'\n' for line in data])

**Encode/Decode functions**

In [ ]:
def encode(data):
  return [char_to_idx[character] for character in data]

def decode(data):
  return ''.join([idx_to_char[idx] for idx in data])

**Dataset function**

In [ ]:
def create_dataset(data=data, test=False):

  data = torch.tensor(encode(data))
  if test:
    n = int(0.9*len(data))
    X_train, X_test = data[:n], data[n:]
    return X_train, X_test
  else:
    X_train = data
    return X_train

**Get batch function**

In [ ]:
def get_batch(data):
    idx = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE, 1))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in idx])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in idx])
    x, y = x.to(DEVICE), y.to(DEVICE)
    return x, y

**Hyperparameters**

In [ ]:
BATCH_SIZE = 32
BLOCK_SIZE = 256
MAX_ITERS = 3000
LR = 3e-4
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
EMB_SIZE = 576
HEAD_NUM = 6
HEAD_SIZE = 96
LAYERS_NUM = 6
DROPOUT = 0.2

**Self attention**

In [ ]:
class Attention(nn.Module):

    def __init__(self, HEAD_SIZE):
        super().__init__()
        self.key = nn.Linear(EMB_SIZE, HEAD_SIZE, bias=False)
        self.query = nn.Linear(EMB_SIZE, HEAD_SIZE, bias=False)
        self.value = nn.Linear(EMB_SIZE, HEAD_SIZE, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        scores = q @ k.transpose(-2,-1) * C**-0.5
        scores = scores.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        scores = F.softmax(scores, dim=-1)

        v = self.value(x)
        out = scores @ v
        return out

**Multilayer perceptron**

In [ ]:
class MLP(nn.Module):

    def __init__(self, EMB_SIZE):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(EMB_SIZE, 4 * EMB_SIZE),
            nn.ReLU(),
            nn.Linear(4 * EMB_SIZE, EMB_SIZE),
            nn.Dropout(DROPOUT),
        )

    def forward(self, x):
        return self.net(x)

**Block**

In [ ]:
class Block(nn.Module):

    def __init__(self, EMB_SIZE, HEADS_NUM):
        super().__init__()

        self.heads = nn.ModuleList([Attention(HEAD_SIZE) for _ in range(HEADS_NUM)])
        self.projection = nn.Linear(HEAD_SIZE * HEADS_NUM, EMB_SIZE)

        self.mlp = MLP(EMB_SIZE)
        self.ln1 = nn.LayerNorm(EMB_SIZE)
        self.ln2 = nn.LayerNorm(EMB_SIZE)

    def forward(self, x):
        x = x + torch.cat([h(self.ln1(x)) for h in self.heads], dim=-1)
        x = self.projection(x)
        x = x + self.mlp(self.ln2(x))
        return x

**GPT Model**

In [ ]:
class GPT(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, EMB_SIZE)
        self.position_embedding_table = nn.Embedding(BLOCK_SIZE, EMB_SIZE)
        self.blocks = nn.Sequential(*[Block(EMB_SIZE, HEAD_NUM) for _ in range(LAYERS_NUM)])
        self.ln_f = nn.LayerNorm(EMB_SIZE)
        self.lm_head = nn.Linear(EMB_SIZE, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        position_emb = self.position_embedding_table(torch.arange(T, device=DEVICE))
        x = token_emb + position_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):

        for _ in range(max_new_tokens):
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)

        return idx

**Create dataset**

In [ ]:
X_train, X_test = create_dataset(test=True)

**Create model**

In [ ]:
model = GPT()
m = model.to(DEVICE)

**Check number of parameters**

In [ ]:
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

24.164433 M parameters


**Train the model**

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=LR)

In [ ]:
for iter in range(MAX_ITERS):
    xb, yb = get_batch(X_train)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if (iter+1) % 250 == 0:
      test_loss = []
      for _ in range(100):
        xb_test, yb_test = get_batch(X_test)
        logits, loss = m(xb_test, yb_test)
        test_loss.append(loss.item())
      print(f'Test loss for the {iter+1}th iter is: {sum(test_loss) / len(test_loss)}')


**Generate from the model**

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=DEVICE)
print(decode(m.generate(context, max_new_tokens=5000)[0].tolist()))